In [12]:
import os
import numpy as np
import pandas as pd

# Charger les embeddings textuels (TF-IDF)
text_embedding_file = os.path.join("..", "data", "tfidf_embeddings.npz")
text_data = np.load(text_embedding_file, allow_pickle=True)  # Charger avec allow_pickle=True
text_embeddings = text_data['embeddings']
product_ids_text = text_data['product_id']

# Afficher les dimensions des embeddings textuels
print(f"Dimensions des embeddings textuels : {text_embeddings.shape}")


Dimensions des embeddings textuels : (1050, 5317)


In [14]:
# Charger les données des vraies catégories
file_path = os.path.join("..", "data", "preprocessed_product_data.csv")
data = pd.read_csv(file_path)

# Vérifier les premières lignes et la colonne des vraies catégories
print(data.head())
print(f"Colonnes disponibles : {data.columns}")


                            uniq_id            crawl_timestamp  \
0  55b85ea15a1536d46b7190ad6fff8ce7  2016-04-30 03:22:56 +0000   
1  7b72c92c2f6c40268628ec5f14c6d590  2016-04-30 03:22:56 +0000   
2  64d5d4a258243731dc7bbb1eef49ad74  2016-04-30 03:22:56 +0000   
3  d4684dcdc759dd9cdf41504698d737d8  2016-06-20 08:49:52 +0000   
4  6325b6870c54cd47be6ebfbffa620ec7  2016-06-20 08:49:52 +0000   

                                         product_url  \
0  http://www.flipkart.com/elegance-polyester-mul...   
1  http://www.flipkart.com/sathiyas-cotton-bath-t...   
2  http://www.flipkart.com/eurospa-cotton-terry-f...   
3  http://www.flipkart.com/santosh-royal-fashion-...   
4  http://www.flipkart.com/jaipur-print-cotton-fl...   

                                        product_name  \
0  Elegance Polyester Multicolor Abstract Eyelet ...   
1                         Sathiyas Cotton Bath Towel   
2                Eurospa Cotton Terry Face Towel Set   
3  SANTOSH ROYAL FASHION Cotton Printed Ki

In [13]:
# Charger les embeddings visuels (VGG16)
visual_embedding_file = os.path.join("..", "models", "best_model2_vgg16_embeddings.npz")
visual_data = np.load(visual_embedding_file, allow_pickle=True)  # Charger avec allow_pickle=True
visual_embeddings = visual_data['embeddings']
product_ids_visual = visual_data['product_id']

# Afficher les dimensions des embeddings visuels
print(f"Dimensions des embeddings visuels : {visual_embeddings.shape}")


Dimensions des embeddings visuels : (210, 7)


In [15]:
# Vérification des identifiants communs
common_ids = np.intersect1d(product_ids_text, product_ids_visual.astype(str))
print(f"Nombre d'identifiants communs : {len(common_ids)}")

# Afficher quelques identifiants communs pour vérification
print(f"Premiers identifiants communs : {common_ids[:5]}")


Nombre d'identifiants communs : 0
Premiers identifiants communs : []


In [16]:
# Filtrer les embeddings textuels et visuels en fonction des identifiants communs
text_embeddings_filtered = text_embeddings[np.isin(product_ids_text, common_ids)]
visual_embeddings_filtered = visual_embeddings[np.isin(product_ids_visual.astype(str), common_ids)]

# Vérifier les dimensions après filtrage
print(f"Dimensions des embeddings textuels filtrés : {text_embeddings_filtered.shape}")
print(f"Dimensions des embeddings visuels filtrés : {visual_embeddings_filtered.shape}")


Dimensions des embeddings textuels filtrés : (0, 5317)
Dimensions des embeddings visuels filtrés : (0, 7)


In [17]:
# Concaténer les embeddings textuels et visuels
combined_embeddings = np.concatenate([text_embeddings_filtered, visual_embeddings_filtered], axis=1)

# Vérifier les dimensions des embeddings combinés
print(f"Dimensions des embeddings combinés : {combined_embeddings.shape}")


Dimensions des embeddings combinés : (0, 5324)


In [18]:
from sklearn.cluster import KMeans
from sklearn import metrics

# Nombre de clusters basé sur le nombre de catégories réelles
num_clusters = len(np.unique(data['category_num']))
print(f"Nombre de clusters : {num_clusters}")

# Appliquer KMeans
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(combined_embeddings)

# Labels prédits par KMeans
predicted_labels = kmeans.labels_

# Vrais labels (catégories réelles)
true_labels = data['category_num'].values


Nombre de clusters : 7


ValueError: Found array with 0 sample(s) (shape=(0, 5324)) while a minimum of 1 is required by KMeans.

In [19]:
# Calculer le score ARI (Adjusted Rand Index)
ari_score = metrics.adjusted_rand_score(true_labels, predicted_labels)
print(f"ARI Score for combined text + image embeddings: {ari_score:.4f}")


NameError: name 'true_labels' is not defined

In [20]:
# Enregistrer les embeddings concaténés
output_file_combined = os.path.join("..", "data", "combined_text_image_embeddings.npz")
np.savez_compressed(output_file_combined, product_id=common_ids, embeddings=combined_embeddings)

print(f"Combined embeddings have been saved to: {output_file_combined}")


Combined embeddings have been saved to: ..\data\combined_text_image_embeddings.npz


In [9]:
import numpy as np

# Charger le fichier npz
file_path = "../models/best_model2_vgg16_embeddings.npz"  # Modifiez le chemin si nécessaire
data = np.load(file_path)

# Afficher les clés disponibles dans le fichier
print("Clés disponibles dans le fichier .npz :", data.files)

# Visualiser les premières lignes des embeddings et des identifiants
product_ids = data['product_id']
embeddings = data['embeddings']

# Affichage des premières lignes
print(f"Premiers identifiants de produits : {product_ids[:5]}")
print(f"Premiers embeddings (dimensions) : {embeddings[:5]}")


Clés disponibles dans le fichier .npz : ['product_id', 'embeddings']
Premiers identifiants de produits : [0 1 2 3 4]
Premiers embeddings (dimensions) : [[2.6755673e-01 7.6281163e-03 6.6825360e-01 1.0395801e-03 4.7152596e-08
  5.5521332e-02 6.4392117e-07]
 [2.1867720e-12 6.2299409e-11 4.1947531e-22 1.0000000e+00 1.0719335e-11
  1.5008108e-16 1.3913483e-14]
 [1.9252884e-14 1.7274643e-21 4.4928514e-22 9.4807976e-21 2.8298977e-24
  9.1848936e-22 1.0000000e+00]
 [3.0350892e-04 4.7364594e-08 8.0512708e-09 9.9969482e-01 1.5984339e-08
  1.6652419e-06 4.6188102e-13]
 [8.4365986e-04 5.2883530e-01 4.0030654e-09 4.5807463e-01 7.8298029e-10
  1.2206303e-02 4.0079325e-05]]


In [11]:
# Vérification des identifiants communs
common_ids = np.intersect1d(product_ids_text, product_ids_visual.astype(str))
print(f"Nombre d'identifiants communs : {len(common_ids)}")


Nombre d'identifiants communs : 0


In [10]:
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
import pandas as pd

# Charger les données pour les vraies catégories
file_path = os.path.join("..", "data", "preprocessed_product_data.csv")
data = pd.read_csv(file_path)

# Charger les embeddings textuels (TF-IDF)
text_embedding_file = os.path.join("..", "data", "tfidf_embeddings.npz")
text_data = np.load(text_embedding_file, allow_pickle=True)  # Charger avec allow_pickle=True
text_embeddings = text_data['embeddings']
product_ids_text = text_data['product_id']

# Charger les embeddings visuels (VGG16 - modèle 2)
visual_embedding_file = os.path.join("..", "models", "best_model2_vgg16_embeddings.npz")
visual_data = np.load(visual_embedding_file, allow_pickle=True)  # Charger avec allow_pickle=True
visual_embeddings = visual_data['embeddings']
product_ids_visual = visual_data['product_id']

# Solution 2: Réaligner les identifiants de produits
common_ids = np.intersect1d(product_ids_text, product_ids_visual.astype(str))  # Identifiants communs

# Filtrer les embeddings textuels et visuels en fonction des identifiants communs
text_embeddings_filtered = text_embeddings[np.isin(product_ids_text, common_ids)]
visual_embeddings_filtered = visual_embeddings[np.isin(product_ids_visual.astype(str), common_ids)]

# Concaténer les embeddings textuels et visuels
combined_embeddings = np.concatenate([text_embeddings_filtered, visual_embeddings_filtered], axis=1)

# Appliquer KMeans sur les embeddings combinés
num_clusters = len(np.unique(data['category_num']))  # Nombre de clusters selon les catégories réelles
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(combined_embeddings)

# Labels prédits par KMeans
predicted_labels = kmeans.labels_

# Vrais labels (catégories réelles)
true_labels = data['category_num'].values  # Les vraies catégories doivent être dans votre dataframe

# Calculer le score ARI (Adjusted Rand Index)
ari_score = metrics.adjusted_rand_score(true_labels, predicted_labels)
print(f"ARI Score for combined text + image embeddings: {ari_score:.4f}")

# Enregistrer les embeddings concaténés
output_file_combined = os.path.join("..", "data", "combined_text_image_embeddings.npz")
np.savez_compressed(output_file_combined, product_id=common_ids, embeddings=combined_embeddings)
print(f"Combined embeddings have been saved to: {output_file_combined}")


ValueError: Found array with 0 sample(s) (shape=(0, 5324)) while a minimum of 1 is required by KMeans.

In [5]:
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
import pandas as pd

# Charger les données pour les vraies catégories
file_path = os.path.join("..", "data", "preprocessed_product_data.csv")
data = pd.read_csv(file_path)

# Charger les embeddings textuels (TF-IDF)
text_embedding_file = os.path.join("..", "data", "tfidf_embeddings.npz")
text_data = np.load(text_embedding_file, allow_pickle=True)  # Charger avec allow_pickle=True
text_embeddings = text_data['embeddings']
product_ids_text = text_data['product_id']

# Charger les embeddings visuels (VGG16 - modèle 2)
visual_embedding_file = os.path.join("..", "models", "best_model2_vgg16_embeddings.npz")
visual_data = np.load(visual_embedding_file, allow_pickle=True)  # Charger avec allow_pickle=True
visual_embeddings = visual_data['embeddings']
product_ids_visual = visual_data['product_id']

# Solution 2: Réaligner les identifiants de produits
common_ids = np.intersect1d(product_ids_text, product_ids_visual.astype(str))  # Identifiants communs

# Filtrer les embeddings textuels et visuels en fonction des identifiants communs
text_embeddings_filtered = text_embeddings[np.isin(product_ids_text, common_ids)]
visual_embeddings_filtered = visual_embeddings[np.isin(product_ids_visual.astype(str), common_ids)]

# Concaténer les embeddings textuels et visuels
combined_embeddings = np.concatenate([text_embeddings_filtered, visual_embeddings_filtered], axis=1)

# Appliquer KMeans sur les embeddings combinés
num_clusters = len(np.unique(data['category_num']))  # Nombre de clusters selon les catégories réelles
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(combined_embeddings)

# Labels prédits par KMeans
predicted_labels = kmeans.labels_

# Vrais labels (catégories réelles)
true_labels = data['category_num'].values  # Les vraies catégories doivent être dans votre dataframe

# Calculer le score ARI (Adjusted Rand Index)
ari_score = metrics.adjusted_rand_score(true_labels, predicted_labels)
print(f"ARI Score for combined text + image embeddings: {ari_score:.4f}")

# Enregistrer les embeddings concaténés
output_file_combined = os.path.join("..", "data", "combined_text_image_embeddings.npz")
np.savez_compressed(output_file_combined, product_id=common_ids, embeddings=combined_embeddings)
print(f"Combined embeddings have been saved to: {output_file_combined}")


ValueError: Found array with 0 sample(s) (shape=(0, 5324)) while a minimum of 1 is required by KMeans.

In [6]:
common_ids = np.intersect1d(product_ids_text, product_ids_visual.astype(str))
print(f"Nombre d'identifiants communs : {len(common_ids)}")


Nombre d'identifiants communs : 0


In [7]:
text_embeddings_filtered = text_embeddings[np.isin(product_ids_text, common_ids)]
visual_embeddings_filtered = visual_embeddings[np.isin(product_ids_visual.astype(str), common_ids)]

print(f"Dimensions des embeddings textuels filtrés : {text_embeddings_filtered.shape}")
print(f"Dimensions des embeddings visuels filtrés : {visual_embeddings_filtered.shape}")


Dimensions des embeddings textuels filtrés : (0, 5317)
Dimensions des embeddings visuels filtrés : (0, 7)


In [8]:
print(f"Premiers identifiants textuels : {product_ids_text[:5]}")
print(f"Premiers identifiants visuels : {product_ids_visual[:5]}")


Premiers identifiants textuels : ['55b85ea15a1536d46b7190ad6fff8ce7' '7b72c92c2f6c40268628ec5f14c6d590'
 '64d5d4a258243731dc7bbb1eef49ad74' 'd4684dcdc759dd9cdf41504698d737d8'
 '6325b6870c54cd47be6ebfbffa620ec7']
Premiers identifiants visuels : [0 1 2 3 4]


In [2]:
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
import pandas as pd

# Charger les données pour les vraies catégories
file_path = os.path.join("..", "data", "preprocessed_product_data.csv")
data = pd.read_csv(file_path)

# Charger les embeddings textuels (TF-IDF)
text_embedding_file = os.path.join("..", "data", "tfidf_embeddings.npz")
text_data = np.load(text_embedding_file, allow_pickle=True)  # Charger avec allow_pickle=True
text_embeddings = text_data['embeddings']
product_ids_text = text_data['product_id']

# Charger les embeddings visuels (VGG16 - modèle 2)
visual_embedding_file = os.path.join("..", "models", "best_model2_vgg16_embeddings.npz")
visual_data = np.load(visual_embedding_file, allow_pickle=True)  # Charger avec allow_pickle=True
visual_embeddings = visual_data['embeddings']
product_ids_visual = visual_data['product_id']

# Vérifier que les identifiants de produits sont dans le même ordre (ou les réaligner)
assert np.array_equal(product_ids_text, product_ids_visual), "Les identifiants de produits ne correspondent pas entre les deux embeddings !"

# Concaténer les embeddings textuels et visuels
combined_embeddings = np.concatenate([text_embeddings, visual_embeddings], axis=1)

# Appliquer KMeans sur les embeddings combinés
num_clusters = len(np.unique(data['category_num']))  # Nombre de clusters selon les catégories réelles
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(combined_embeddings)

# Labels prédits par KMeans
predicted_labels = kmeans.labels_

# Vrais labels (catégories réelles)
true_labels = data['category_num'].values  # Les vraies catégories doivent être dans votre dataframe

# Calculer le score ARI (Adjusted Rand Index)
ari_score = metrics.adjusted_rand_score(true_labels, predicted_labels)
print(f"ARI Score for combined text + image embeddings: {ari_score:.4f}")

# Enregistrer les embeddings concaténés
output_file_combined = os.path.join("..", "data", "combined_text_image_embeddings.npz")
np.savez_compressed(output_file_combined, product_id=product_ids_text, embeddings=combined_embeddings)
print(f"Combined embeddings have been saved to: {output_file_combined}")


AssertionError: Les identifiants de produits ne correspondent pas entre les deux embeddings !

In [4]:
print("Product IDs (Text):", product_ids_text)
print("Product IDs (Visual):", product_ids_visual)


Product IDs (Text): ['55b85ea15a1536d46b7190ad6fff8ce7' '7b72c92c2f6c40268628ec5f14c6d590'
 '64d5d4a258243731dc7bbb1eef49ad74' ... '5912e037d12774bb73a2048f35a00009'
 'c3edc504d1b4f0ba6224fa53a43a7ad6' 'f2f027ad6a6df617c9f125173da71e44']
Product IDs (Visual): [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 1

In [1]:
# import os
# import numpy as np
# from sklearn.cluster import KMeans
# from sklearn import metrics
# import pandas as pd
# 
# # Charger les données pour les vraies catégories
# file_path = os.path.join("..", "data", "preprocessed_product_data.csv")
# data = pd.read_csv(file_path)
# 
# # Charger les embeddings textuels (TF-IDF)
# text_embedding_file = os.path.join("..", "data", "tfidf_embeddings.npz")
# text_data = np.load(text_embedding_file)
# text_embeddings = text_data['embeddings']
# product_ids_text = text_data['product_id']
# 
# # Charger les embeddings visuels (VGG16 - modèle 2)
# visual_embedding_file = os.path.join("..", "models", "best_model2_vgg16_embeddings.npz")  # Chemin vers les embeddings VGG16 modèle 2
# visual_data = np.load(visual_embedding_file)
# visual_embeddings = visual_data['embeddings']
# product_ids_visual = visual_data['product_id']
# 
# # Vérifier que les identifiants de produits sont dans le même ordre (ou les réaligner)
# assert np.array_equal(product_ids_text, product_ids_visual), "Les identifiants de produits ne correspondent pas entre les deux embeddings !"
# 
# # Concaténer les embeddings textuels et visuels
# combined_embeddings = np.concatenate([text_embeddings, visual_embeddings], axis=1)
# 
# # Appliquer KMeans sur les embeddings combinés
# num_clusters = len(np.unique(data['category_num']))  # Nombre de clusters selon les catégories réelles
# kmeans = KMeans(n_clusters=num_clusters, random_state=42)
# kmeans.fit(combined_embeddings)
# 
# # Labels prédits par KMeans
# predicted_labels = kmeans.labels_
# 
# # Vrais labels (catégories réelles)
# true_labels = data['category_num'].values  # Les vraies catégories doivent être dans votre dataframe
# 
# # Calculer le score ARI (Adjusted Rand Index)
# ari_score = metrics.adjusted_rand_score(true_labels, predicted_labels)
# print(f"ARI Score for combined text + image embeddings: {ari_score:.4f}")
# 
# # Enregistrer les embeddings concaténés
# output_file_combined = os.path.join("..", "data", "combined_text_image_embeddings.npz")
# np.savez_compressed(output_file_combined, product_id=product_ids_text, embeddings=combined_embeddings)
# print(f"Combined embeddings have been saved to: {output_file_combined}")


ValueError: Object arrays cannot be loaded when allow_pickle=False